In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import skimage.io as io
import os, sys
from PIL import Image

# Use second GPU -- change if you want to use a first one


# Add a path to a custom fork of TF-Slim
# Get it from here:
# https://github.com/warmspringwinds/models/tree/fully_conv_vgg
sys.path.append("/home/shou/network/tf-models/research/slim")

# Add path to the cloned library
sys.path.append("/home/shou/network/fcn/tf-image-segmentation")

checkpoints_dir = '/home/shou/network/checkpoints'
log_folder = '/home/shou/network/log_folder_fcn8s'

slim = tf.contrib.slim

from tf_image_segmentation.utils.tf_records import read_tfrecord_and_decode_into_image_annotation_pair_tensors
from tf_image_segmentation.models.fcn_8s import FCN_8s

from tf_image_segmentation.utils.pascal_voc import pascal_segmentation_lut

from tf_image_segmentation.utils.training import get_valid_logits_and_labels

from tf_image_segmentation.utils.augmentation import (distort_randomly_image_color,
                                                      flip_randomly_left_right_image_with_annotation,
                                                      scale_randomly_image_with_annotation_with_fixed_size_output)

In [3]:
image_train_size = [384, 384]
number_of_classes = 2
tfrecord_filename = '/home/shou/network/dataset/pascal_augmented_train_island.tfrecords'
pascal_voc_lut = pascal_segmentation_lut(number_of_classes)
class_labels = pascal_voc_lut.keys()

fcn_16s_checkpoint_path = '/home/shou/network/dataset/model_fcn16s_final.ckpt'

filename_queue = tf.train.string_input_producer(
    [tfrecord_filename], num_epochs=5)

image, annotation = read_tfrecord_and_decode_into_image_annotation_pair_tensors(filename_queue)

# Various data augmentation stages
image, annotation = flip_randomly_left_right_image_with_annotation(image, annotation)

# image = distort_randomly_image_color(image)

resized_image, resized_annotation = scale_randomly_image_with_annotation_with_fixed_size_output(image, annotation, image_train_size)


resized_annotation = tf.squeeze(resized_annotation)

image_batch, annotation_batch = tf.train.shuffle_batch( [resized_image, resized_annotation],
                                             batch_size=1,
                                             capacity=3000,
                                             num_threads=2,
                                             min_after_dequeue=1000)

upsampled_logits_batch, fcn_16s_variables_mapping = FCN_8s(image_batch_tensor=image_batch,
                                                           number_of_classes=number_of_classes,
                                                           is_training=True)


valid_labels_batch_tensor, valid_logits_batch_tensor = get_valid_logits_and_labels(annotation_batch_tensor=annotation_batch,
                                                                                     logits_batch_tensor=upsampled_logits_batch,
                                                                                    class_labels=class_labels)



cross_entropies = tf.nn.softmax_cross_entropy_with_logits(logits=valid_logits_batch_tensor,
                                                          labels=valid_labels_batch_tensor)

#cross_entropy_sum = tf.reduce_sum(cross_entropies)

cross_entropy_sum = tf.reduce_mean(cross_entropies)

pred = tf.argmax(upsampled_logits_batch, dimension=3)

probabilities = tf.nn.softmax(upsampled_logits_batch)

Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
fn1/fn2 are deprecated in favor of the true_fn/false_fn arguments.
Instructions for updating:
Use the `axis` argument instead


In [4]:
with tf.variable_scope("adam_vars"):
    train_step = tf.train.AdamOptimizer(learning_rate=0.000000001).minimize(cross_entropy_sum)


#adam_optimizer_variables = slim.get_variables_to_restore(include=['adam_vars'])

# Variable's initialization functions
init_fn = slim.assign_from_checkpoint_fn(model_path=fcn_16s_checkpoint_path,
                                         var_list=fcn_16s_variables_mapping)

global_vars_init_op = tf.global_variables_initializer()

tf.summary.scalar('cross_entropy_loss', cross_entropy_sum)

merged_summary_op = tf.summary.merge_all()

summary_string_writer = tf.summary.FileWriter(log_folder)

# Create the log folder if doesn't exist yet
if not os.path.exists(log_folder):
     os.makedirs(log_folder)

#optimization_variables_initializer = tf.variables_initializer(adam_optimizer_variables)
    
#The op for initializing the variables.
local_vars_init_op = tf.local_variables_initializer()

combined_op = tf.group(local_vars_init_op, global_vars_init_op)

# We need this to save only model variables and omit
# optimization-related and other variables.
model_variables = slim.get_model_variables()
saver = tf.train.Saver(model_variables)

In [5]:
with tf.Session()  as sess:
    
    sess.run(combined_op)
    init_fn(sess)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # Let's read off 3 batches just for example
    for i in xrange(486 * 5):
    
        cross_entropy, summary_string, _ = sess.run([ cross_entropy_sum,
                                                      merged_summary_op,
                                                      train_step ])

        summary_string_writer.add_summary(summary_string, 486 * 10 + i)
        
        print("step :" + str(i) + " Loss: " + str(cross_entropy))
        
        if i % 486 == 0:
            save_path = saver.save(sess, "/home/shou/network/dataset/model_fcn8s_final.ckpt")
            print("Model saved in file: %s" % save_path)
            
        
    coord.request_stop()
    coord.join(threads)
    
    save_path = saver.save(sess, "/home/shou/network/dataset/model_fcn8s_final.ckpt")
    print("Model saved in file: %s" % save_path)
    
summary_string_writer.close()

INFO:tensorflow:Restoring parameters from /home/shou/network/dataset/model_fcn16s_final.ckpt
step :0 Loss: 0.109686
Model saved in file: /home/shou/network/dataset/model_fcn8s_final.ckpt
step :1 Loss: 0.109197
step :2 Loss: 0.0807109
step :3 Loss: 0.0777678
step :4 Loss: 0.0934385
step :5 Loss: 0.044567
step :6 Loss: 0.201067
step :7 Loss: 0.0962793
step :8 Loss: 0.155758
step :9 Loss: 0.160842
step :10 Loss: 0.15019
step :11 Loss: 0.162537
step :12 Loss: 0.130364
step :13 Loss: 0.157157
step :14 Loss: 0.112052
step :15 Loss: 0.133374
step :16 Loss: 0.143589
step :17 Loss: 0.0891468
step :18 Loss: 0.0960048
step :19 Loss: 0.184017
step :20 Loss: 0.132332
step :21 Loss: 0.158992
step :22 Loss: 0.198495
step :23 Loss: 0.131338
step :24 Loss: 0.0379294
step :25 Loss: 0.12577
step :26 Loss: 0.0718431
step :27 Loss: 0.154613
step :28 Loss: 0.217096
step :29 Loss: 0.161405
step :30 Loss: 0.0904403
step :31 Loss: 0.0555786
step :32 Loss: 0.0865612
step :33 Loss: 0.0774507
step :34 Loss: 0.121

step :323 Loss: 0.181175
step :324 Loss: 0.150646
step :325 Loss: 0.135598
step :326 Loss: 0.210695
step :327 Loss: 0.131739
step :328 Loss: 0.143778
step :329 Loss: 0.0807763
step :330 Loss: 0.123752
step :331 Loss: 0.0512108
step :332 Loss: 0.0814593
step :333 Loss: 0.164825
step :334 Loss: 0.0977962
step :335 Loss: 0.13645
step :336 Loss: 0.159602
step :337 Loss: 0.149109
step :338 Loss: 0.0972088
step :339 Loss: 0.0742145
step :340 Loss: 0.107283
step :341 Loss: 0.0932185
step :342 Loss: 0.080807
step :343 Loss: 0.0803085
step :344 Loss: 0.0932037
step :345 Loss: 0.150645
step :346 Loss: 0.10428
step :347 Loss: 0.151033
step :348 Loss: 0.111929
step :349 Loss: 0.122784
step :350 Loss: 0.155318
step :351 Loss: 0.0903587
step :352 Loss: 0.128373
step :353 Loss: 0.103495
step :354 Loss: 0.104344
step :355 Loss: 0.26093
step :356 Loss: 0.0584579
step :357 Loss: 0.179274
step :358 Loss: 0.180735
step :359 Loss: 0.193826
step :360 Loss: 0.0992287
step :361 Loss: 0.111282
step :362 Loss: 

step :645 Loss: 0.0617564
step :646 Loss: 0.0926444
step :647 Loss: 0.392855
step :648 Loss: 0.105298
step :649 Loss: 0.0658216
step :650 Loss: 0.0264588
step :651 Loss: 0.0988386
step :652 Loss: 0.0974738
step :653 Loss: 0.112532
step :654 Loss: 0.120634
step :655 Loss: 0.0917438
step :656 Loss: 0.100884
step :657 Loss: 0.10751
step :658 Loss: 0.323577
step :659 Loss: 0.22428
step :660 Loss: 0.234202
step :661 Loss: 0.160397
step :662 Loss: 0.137665
step :663 Loss: 0.158973
step :664 Loss: 0.176385
step :665 Loss: 0.113487
step :666 Loss: 0.13486
step :667 Loss: 0.0701581
step :668 Loss: 0.114325
step :669 Loss: 0.162177
step :670 Loss: 0.0980463
step :671 Loss: 0.22798
step :672 Loss: 0.0574754
step :673 Loss: 0.0680944
step :674 Loss: 0.0482682
step :675 Loss: 0.098393
step :676 Loss: 0.0913149
step :677 Loss: 0.140912
step :678 Loss: 0.0990581
step :679 Loss: 0.113615
step :680 Loss: 0.151019
step :681 Loss: 0.0470667
step :682 Loss: 0.0155081
step :683 Loss: 0.0677532
step :684 Lo

step :970 Loss: 0.0837784
step :971 Loss: 0.132962
step :972 Loss: 0.222037
Model saved in file: /home/shou/network/dataset/model_fcn8s_final.ckpt
step :973 Loss: 0.152095
step :974 Loss: 0.132157
step :975 Loss: 0.130765
step :976 Loss: 0.151212
step :977 Loss: 0.116721
step :978 Loss: 0.0682985
step :979 Loss: 0.102206
step :980 Loss: 0.136084
step :981 Loss: 0.0766607
step :982 Loss: 0.103454
step :983 Loss: 0.0395665
step :984 Loss: 0.0827084
step :985 Loss: 0.173528
step :986 Loss: 0.114488
step :987 Loss: 0.211213
step :988 Loss: 0.193318
step :989 Loss: 0.0935314
step :990 Loss: 0.195075
step :991 Loss: 0.132398
step :992 Loss: 0.210732
step :993 Loss: 0.105158
step :994 Loss: 0.0640544
step :995 Loss: 0.118787
step :996 Loss: 0.150904
step :997 Loss: 0.124493
step :998 Loss: 0.0980743
step :999 Loss: 0.0953046
step :1000 Loss: 0.166581
step :1001 Loss: 0.220533
step :1002 Loss: 0.260649
step :1003 Loss: 0.0229348
step :1004 Loss: 0.097039
step :1005 Loss: 0.0175893
step :1006 L

step :1281 Loss: 0.103953
step :1282 Loss: 0.136192
step :1283 Loss: 0.258043
step :1284 Loss: 0.230416
step :1285 Loss: 0.124164
step :1286 Loss: 0.150596
step :1287 Loss: 0.145064
step :1288 Loss: 0.1598
step :1289 Loss: 0.158132
step :1290 Loss: 0.163492
step :1291 Loss: 0.110396
step :1292 Loss: 0.216246
step :1293 Loss: 0.152851
step :1294 Loss: 0.0903331
step :1295 Loss: 0.138666
step :1296 Loss: 0.140571
step :1297 Loss: 0.0736295
step :1298 Loss: 0.140802
step :1299 Loss: 0.167558
step :1300 Loss: 0.0770572
step :1301 Loss: 0.145615
step :1302 Loss: 0.135288
step :1303 Loss: 0.0974527
step :1304 Loss: 0.13711
step :1305 Loss: 0.0715674
step :1306 Loss: 0.138077
step :1307 Loss: 0.136125
step :1308 Loss: 0.0930995
step :1309 Loss: 0.138643
step :1310 Loss: 0.121257
step :1311 Loss: 0.148059
step :1312 Loss: 0.15063
step :1313 Loss: 0.184187
step :1314 Loss: 0.135883
step :1315 Loss: 0.137403
step :1316 Loss: 0.108983
step :1317 Loss: 0.117887
step :1318 Loss: 0.143323
step :1319

step :1591 Loss: 0.0747982
step :1592 Loss: 0.0769119
step :1593 Loss: 0.123229
step :1594 Loss: 0.107513
step :1595 Loss: 0.149902
step :1596 Loss: 0.110943
step :1597 Loss: 0.144352
step :1598 Loss: 0.0986694
step :1599 Loss: 0.0729545
step :1600 Loss: 0.169871
step :1601 Loss: 0.0167788
step :1602 Loss: 0.1657
step :1603 Loss: 0.11362
step :1604 Loss: 0.045865
step :1605 Loss: 0.11849
step :1606 Loss: 0.145366
step :1607 Loss: 0.193167
step :1608 Loss: 0.0794194
step :1609 Loss: 0.175328
step :1610 Loss: 0.166751
step :1611 Loss: 0.264926
step :1612 Loss: 0.0962313
step :1613 Loss: 0.157164
step :1614 Loss: 0.177723
step :1615 Loss: 0.110814
step :1616 Loss: 0.144102
step :1617 Loss: 0.0937548
step :1618 Loss: 0.176764
step :1619 Loss: 0.097874
step :1620 Loss: 0.0856891
step :1621 Loss: 0.125061
step :1622 Loss: 0.162488
step :1623 Loss: 0.187503
step :1624 Loss: 0.148682
step :1625 Loss: 0.0882031
step :1626 Loss: 0.149281
step :1627 Loss: 0.116653
step :1628 Loss: 0.118226
step :

step :1904 Loss: 0.113627
step :1905 Loss: 0.171644
step :1906 Loss: 0.133018
step :1907 Loss: 0.151123
step :1908 Loss: 0.211166
step :1909 Loss: 0.144559
step :1910 Loss: 0.0633918
step :1911 Loss: 0.07115
step :1912 Loss: 0.0409409
step :1913 Loss: 0.0829662
step :1914 Loss: 0.142824
step :1915 Loss: 0.106947
step :1916 Loss: 0.0858575
step :1917 Loss: 0.121607
step :1918 Loss: 0.118176
step :1919 Loss: 0.130002
step :1920 Loss: 0.101261
step :1921 Loss: 0.0978236
step :1922 Loss: 0.0459107
step :1923 Loss: 0.0847273
step :1924 Loss: 0.19087
step :1925 Loss: 0.145902
step :1926 Loss: 0.0775976
step :1927 Loss: 0.0781121
step :1928 Loss: 0.0987315
step :1929 Loss: 0.0374947
step :1930 Loss: 0.0735371
step :1931 Loss: 0.130528
step :1932 Loss: 0.10375
step :1933 Loss: 0.0952303
step :1934 Loss: 0.170627
step :1935 Loss: 0.0943794
step :1936 Loss: 0.0744861
step :1937 Loss: 0.143946
step :1938 Loss: 0.129795
step :1939 Loss: 0.134791
step :1940 Loss: 0.134953
step :1941 Loss: 0.0687209

step :2214 Loss: 0.169438
step :2215 Loss: 0.127888
step :2216 Loss: 0.21838
step :2217 Loss: 0.260696
step :2218 Loss: 0.0994474
step :2219 Loss: 0.190767
step :2220 Loss: 0.0619265
step :2221 Loss: 0.233485
step :2222 Loss: 0.113105
step :2223 Loss: 0.254467
step :2224 Loss: 0.0632499
step :2225 Loss: 0.134701
step :2226 Loss: 0.29545
step :2227 Loss: 0.0616224
step :2228 Loss: 0.0688887
step :2229 Loss: 0.0707431
step :2230 Loss: 0.196106
step :2231 Loss: 0.202141
step :2232 Loss: 0.156423
step :2233 Loss: 0.142786
step :2234 Loss: 0.0861569
step :2235 Loss: 0.116084
step :2236 Loss: 0.076467
step :2237 Loss: 0.0946672
step :2238 Loss: 0.131755
step :2239 Loss: 0.166388
step :2240 Loss: 0.0761301
step :2241 Loss: 0.0852254
step :2242 Loss: 0.0620668
step :2243 Loss: 0.105961
step :2244 Loss: 0.0979922
step :2245 Loss: 0.0963292
step :2246 Loss: 0.0831083
step :2247 Loss: 0.126317
step :2248 Loss: 0.0735016
step :2249 Loss: 0.105779
step :2250 Loss: 0.181947
step :2251 Loss: 0.025623